<a href="https://colab.research.google.com/github/jdasam/ant5015-2023/blob/main/notebooks/7th_week_live_coding-music-generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Music Generation
- Music Generation with MusPy

In [ ]:
'''
install muspy
'''
!pip install muspy

In [ ]:
import muspy
your_path = '~/userdata/datasets/essen_folk/' # Change it!
essen = muspy.EssenFolkSongDatabase(your_path, download_and_extract=True)

In [ ]:
essen.convert()

- You can visualize score with ``muspy.show_score()``
- You can sonify score with ``muspy.synthesize()``

- You need to install font and soundfont
    - ``muspy.download_bravura_font()``
    - ``muspy.download_musescore_soundfont()``

In [ ]:
len(essen)

In [ ]:
a_piece = essen[0]


In [ ]:
type(a_piece)

In [ ]:
import matplotlib.pyplot as plt

muspy.show_score(a_piece, figsize=(30,10), font_scale=200, note_spacing=2)

In [ ]:
audio = a_piece.synthesize()

- You can convert MusPy dataset to pytorch dataset using
    - ``essen.to_pytorch_dataset(representation='note', splits=(0.6, 0.2, 0.2))``
    - this also include train/valid/test split
   


- You can convert representation by using
    - ``muspy.to_pitch_representation``
    - ``muspy.to_pianoroll_representation``
    

## Make Data Loader

In [ ]:
class MelodyDataset:
  def __init__(self, muspy_dataset):
    self.dataset = muspy_dataset
    self.dataset = [x.to_note_representation() for x in self.dataset]
    
    self.idx2pitch, self.idx2dur = self._get_vocab_info()
    
    self.pitch2idx = {x:i for i, x in enumerate(self.idx2pitch)}
    self.dur2idx = {x:i for i, x in enumerate(self.idx2dur)}
    
  def _get_vocab_info(self):
    entire_pitch = []
    entire_dur = []
    for note_rep in self.dataset:
      pitch_in_piece = note_rep[:, 1]
      dur_in_piece = note_rep[:, 2]
      
      entire_pitch += pitch_in_piece.tolist()
      entire_dur += dur_in_piece.tolist()
    return list(set(entire_pitch)), list(set(entire_dur))
    
  def __len__(self):
    return len(self.dataset)
  
  def __getitem__(self, idx):
    note_representation = self.dataset[idx]
#     note_representation = piece.to_note_representation()
    note_representation = note_representation[:,1:3].tolist()
    
    pitch = [self.pitch2idx[pitch_dur[0]] for pitch_dur in note_representation]
    dur = [self.dur2idx[pitch_dur[1]] for pitch_dur in note_representation]
    
    out_tensor = torch.LongTensor([pitch, dur]).T
    
    return out_tensor
    
train_set = MelodyDataset(essen)


## Define Model

## Make Train Loop

## Make Inference
- 